# Data Processing

In [1]:
import os

import numpy as np

import librosa

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
os.chdir('../../')
print(os.getcwd())

/Users/juanpablo/OneDrive/UTS/NNFL/MajorProject


In [3]:
audiopath = 'data/audio/'
words = os.listdir(audiopath)
words.remove('.DS_Store')
words.remove('_background_noise_')

In [4]:
# Re-sampling of the data
def re_sample(wds, tsr):
    audio_samples_rs = []
    labels = []
    for w in wds:
        files = os.listdir(audiopath+w)
        for file in files:
            samples, sample_rate = librosa.load(audiopath+'{}/{}'.format(w, file))
            samples_rs = librosa.resample(samples, sample_rate, tsr)
            audio_samples_rs.append(samples_rs)
            labels.append(w)
    return audio_samples_rs, labels

In [5]:
target_sr = 8000
samples, labels = re_sample(words, target_sr)

In [27]:
# Adds padding to the files shorter than 8000 
X_pad = [np.pad(samples[i], (int(np.floor((target_sr-len(samples[i]))/2)),
                       int(np.ceil((target_sr-len(samples[i]))/2))), 
                'edge') if len(samples[i])<target_sr else samples[i] for i in range(len(samples))]

In [29]:
# Encode class membership vector
def encode_class():
    d_encoded = []
    for i in range(len(labels)):
        membership = np.zeros(len(words))
        membership[words.index(labels[i])] = 1
        d_encoded.append(membership)  
    d_encoded = np.array(d_encoded)
    return d_encoded

<ipython-input-29-e8ef43ef0588>:2: NumbaWarning:


Compilation is falling back to object mode WITH looplifting enabled because Function "encode_class" failed type inference due to: Invalid use of Function(<built-in function array>) with argument(s) of type(s): (list(array(float64, 1d, C)))
 * parameterized
In definition 0:
    TypingError: array(float64, 1d, C) not allowed in a homogeneous sequence
    raised from /Users/juanpablo/Desktop/NNFL/plaidml/lib/python3.7/site-packages/numba/core/typing/npydecl.py:462
In definition 1:
    TypingError: array(float64, 1d, C) not allowed in a homogeneous sequence
    raised from /Users/juanpablo/Desktop/NNFL/plaidml/lib/python3.7/site-packages/numba/core/typing/npydecl.py:462
This error is usually caused by passing an argument of a type that is unsupported by the named function.
[1] During: resolving callee type: Function(<built-in function array>)
[2] During: typing of call at <ipython-input-29-e8ef43ef0588> (9)


File "<ipython-input-29-e8ef43

In [ ]:
d_encoded = encode_class()

In [31]:
# Save the results
np.save('data/samples', X_pad)
np.save('data/labels', d_encoded)